This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
with open('secret_api_key.txt') as secret_key_file:
    API_KEY = secret_key_file.read()

DB_CODE = 'FSE'
DS_CODE = 'AFX_X'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
# API_URL = f'https://www.quandl.com/api/v3/databases/{DB_CODE}.json'
API_URL = f'https://www.quandl.com/api/v3/datasets/{DB_CODE}/{DS_CODE}/data.json'
req = requests.get(API_URL, params={'limit':1}) # 'api_key': API_KEY, 
assert req.status_code == 200, 'API returned status code: %d'%req.status_code

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
req.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
API_URL = f'https://www.quandl.com/api/v3/datasets/{DB_CODE}/{DS_CODE}/data.json'
req = requests.get(API_URL, params={'start_date': '2017-01-01', 'end_date': '2017-12-31'})
# 'api_key': API_KEY,
assert req.status_code == 200, 'API returned status code: %d'%req.status_code
resp = req.json()

In [6]:
## Transpose data, such that each column name is a dictionary key associated with
## a single sequence of values for that column.
dtranspose = list(zip(*resp['dataset_data']['data']))
data = {col_name: dtranspose[idx] for idx, col_name in enumerate(resp['dataset_data']['column_names'])}

In [7]:
## Part 3:
valid_open_data = [data['Open'][i] \
                   for i in range(len(data['Open'])) \
                   if data['Open'][i] is not None]
print('Lowest opening price: $%0.02f'%min(valid_open_data))
print('Highest opening price: $%0.02f'%max(valid_open_data))

Lowest opening price: $34.00
Highest opening price: $53.11


In [8]:
## Part 4:
single_day_changes = [data['High'][i]-data['Low'][i] for i in range(len(data['High']))]
print('Maximum change in a single day: $%0.02f'%max(single_day_changes))

Maximum change in a single day: $2.81


In [9]:
## Part 5
multi_day_changes = [abs(data['Close'][i+1]-data['Close'][i]) for i in range(len(data['Close'])-1)]
print('Maximum change between days: $%0.02f'%max(multi_day_changes))

Maximum change between days: $2.56


In [10]:
## Part 6
avg_volume = sum(data['Traded Volume'])/len(data['Traded Volume'])
print('Mean daily trade volume:', avg_volume)

Mean daily trade volume: 89124.33725490196


In [21]:
## Part 7
sorted_volumes = list(sorted(data['Traded Volume']))
mid = len(sorted_volumes)//2
median_volume = int(sorted_volumes[mid]) if len(sorted_volumes)%2 == 1 else sum(sorted_volumes[mid:mid+2])/2.0
print('Median daily trade volume:', median_volume)

Median daily trade volume: 76286.0
